<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#All-improts-necessary" data-toc-modified-id="All-improts-necessary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>All improts necessary</a></span></li><li><span><a href="#A-few-intro-words" data-toc-modified-id="A-few-intro-words-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>A few intro-words</a></span></li><li><span><a href="#Read-the-data" data-toc-modified-id="Read-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Read the data</a></span></li><li><span><a href="#Add-some-gaps-in-the-data" data-toc-modified-id="Add-some-gaps-in-the-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add some gaps in the data</a></span></li><li><span><a href="#Imputers" data-toc-modified-id="Imputers-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Imputers</a></span><ul class="toc-item"><li><span><a href="#sklearn.impute.SimpleImputer" data-toc-modified-id="sklearn.impute.SimpleImputer-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>sklearn.impute.SimpleImputer</a></span></li></ul></li><li><span><a href="#Scalers" data-toc-modified-id="Scalers-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Scalers</a></span><ul class="toc-item"><li><span><a href="#sklearn.preprocessing.MaxAbsScaler" data-toc-modified-id="sklearn.preprocessing.MaxAbsScaler-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>sklearn.preprocessing.MaxAbsScaler</a></span></li><li><span><a href="#sklearn.preprocessing.MinMaxScaler" data-toc-modified-id="sklearn.preprocessing.MinMaxScaler-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>sklearn.preprocessing.MinMaxScaler</a></span></li><li><span><a href="#sklearn.preprocessing.RobustScaler" data-toc-modified-id="sklearn.preprocessing.RobustScaler-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>sklearn.preprocessing.RobustScaler</a></span></li><li><span><a href="#sklearn.preprocessing.StandardScaler" data-toc-modified-id="sklearn.preprocessing.StandardScaler-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>sklearn.preprocessing.StandardScaler</a></span></li></ul></li><li><span><a href="#Transformers" data-toc-modified-id="Transformers-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Transformers</a></span><ul class="toc-item"><li><span><a href="#sklearn.preprocessing.FunctionTransformer" data-toc-modified-id="sklearn.preprocessing.FunctionTransformer-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>sklearn.preprocessing.FunctionTransformer</a></span></li><li><span><a href="#sklearn.preprocessing.PowerTransformer" data-toc-modified-id="sklearn.preprocessing.PowerTransformer-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>sklearn.preprocessing.PowerTransformer</a></span></li><li><span><a href="#sklearn.preprocessing.QuantileTransformer" data-toc-modified-id="sklearn.preprocessing.QuantileTransformer-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>sklearn.preprocessing.QuantileTransformer</a></span></li></ul></li><li><span><a href="#Encoders" data-toc-modified-id="Encoders-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Encoders</a></span><ul class="toc-item"><li><span><a href="#sklearn.preprocessing.LabelEncoder" data-toc-modified-id="sklearn.preprocessing.LabelEncoder-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>sklearn.preprocessing.LabelEncoder</a></span></li><li><span><a href="#sklearn.preprocessing.LabelBinarizer" data-toc-modified-id="sklearn.preprocessing.LabelBinarizer-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>sklearn.preprocessing.LabelBinarizer</a></span></li><li><span><a href="#sklearn.preprocessing.OneHotEncoder" data-toc-modified-id="sklearn.preprocessing.OneHotEncoder-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>sklearn.preprocessing.OneHotEncoder</a></span></li><li><span><a href="#sklearn.preprocessing.OrdinalEncoder" data-toc-modified-id="sklearn.preprocessing.OrdinalEncoder-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>sklearn.preprocessing.OrdinalEncoder</a></span></li></ul></li><li><span><a href="#Target-balancers" data-toc-modified-id="Target-balancers-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Target balancers</a></span><ul class="toc-item"><li><span><a href="#Under-sampling" data-toc-modified-id="Under-sampling-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Under-sampling</a></span><ul class="toc-item"><li><span><a href="#imblearn.under_sampling.RandomUnderSampler" data-toc-modified-id="imblearn.under_sampling.RandomUnderSampler-9.1.1"><span class="toc-item-num">9.1.1&nbsp;&nbsp;</span>imblearn.under_sampling.RandomUnderSampler</a></span></li><li><span><a href="#imblearn.under_sampling.ClusterCentroids" data-toc-modified-id="imblearn.under_sampling.ClusterCentroids-9.1.2"><span class="toc-item-num">9.1.2&nbsp;&nbsp;</span>imblearn.under_sampling.ClusterCentroids</a></span></li><li><span><a href="#imblearn.under_sampling.NearMiss" data-toc-modified-id="imblearn.under_sampling.NearMiss-9.1.3"><span class="toc-item-num">9.1.3&nbsp;&nbsp;</span>imblearn.under_sampling.NearMiss</a></span></li></ul></li><li><span><a href="#Over-sampling" data-toc-modified-id="Over-sampling-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Over-sampling</a></span><ul class="toc-item"><li><span><a href="#imblearn.over_sampling.RandomOverSampler" data-toc-modified-id="imblearn.over_sampling.RandomOverSampler-9.2.1"><span class="toc-item-num">9.2.1&nbsp;&nbsp;</span>imblearn.over_sampling.RandomOverSampler</a></span></li><li><span><a href="#imblearn.over_sampling.SMOTE" data-toc-modified-id="imblearn.over_sampling.SMOTE-9.2.2"><span class="toc-item-num">9.2.2&nbsp;&nbsp;</span>imblearn.over_sampling.SMOTE</a></span></li><li><span><a href="#imblearn.over_sampling.ADASYN" data-toc-modified-id="imblearn.over_sampling.ADASYN-9.2.3"><span class="toc-item-num">9.2.3&nbsp;&nbsp;</span>imblearn.over_sampling.ADASYN</a></span></li></ul></li></ul></li><li><span><a href="#A-bit-about-the-mess-that-was-created-so-far" data-toc-modified-id="A-bit-about-the-mess-that-was-created-so-far-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>A bit about the mess that was created so far</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

# All improts necessary

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import NearMiss

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
sns.set(font_scale=2)

# A few intro-words

Если в двух словах, то: предобработка важна))

На этом бы можно и завержить рассказ, но это примерно как на вопрос "Как тебя зовут?" ответить "Так, как меня назвали"))

Вообще говоря, под предобработкой данных могут пониматься довольно разные наборы активностей.

Для изображений это может быть масштабирование картинок, их обрезание, наложение различных масок, повороты, смещения и т.д. и т.п.

Для текста это, соответственно, лемматизация, стэмминг, выделение суффиксов и аффиксов, применение различных регулярок.

Для табличных данных своя песня, тут вам и заполнение пропусков в ячейках, и нормализация признаков, и различные линейные\нелинейные преобразования, и квантование, и бинаризация и мнооооого еще чего.

В данном ноуте мы, опять-таки, не будем вгрызаться в глубокие дебри плюсов и минусов приведенных тут техник (хотя, конечно, все относительно)))

Вместо этого пробежимся по некоторым из техник, имплементации для которых содержатся в scikit-learn и imbalanced-learn, с тем, чтобы иметь некое общее представление.

# Read the data

In [ ]:
data = pd.read_csv('../data/datasets/TS_Summer_2018/data.csv', sep=';')

In [ ]:
data.head().T

In [ ]:
data.info()

# Add some gaps in the data

Прикинемся, что в данных есть пропуски.

Для этого возьмем все признаки, кроме целевого и закинем туда охапку nan-ов:

In [ ]:
columns_with_gaps = data.columns[:-1]

In [ ]:
columns_with_gaps

Будем брать процент пропусков в признаке от 0 до 30:

In [ ]:
minimum = 0
maximum = 0.3

Слепим словарь, он нам пригодится для того, чтобы бежать по признакам и закидывать туда пропуски:

In [ ]:
columns_with_gaps_dict = dict(
    zip(
        columns_with_gaps,
        np.random.uniform(
            minimum,
            maximum,
            len(columns_with_gaps)
        )
    )
)

In [ ]:
columns_with_gaps_dict

Теперь, чтобы не попортить исходные данные, возьмем их копию и наполним ее пропусками:

In [ ]:
data_with_gaps_v1 = data.copy()

In [ ]:
for column in columns_with_gaps:
    if columns_with_gaps_dict[column] > 0:
        gaps_count = int(len(data_with_gaps_v1) * columns_with_gaps_dict[column])
        data_with_gaps_v1[column].iloc[
            np.random.randint(
                0,
                len(data_with_gaps_v1),
                gaps_count
            )
        ] = np.nan

In [ ]:
data_with_gaps_v1.info(verbose=True, null_counts=True)

# Imputers

Пропуски, понятное дело, вещь вполне ожидаемая.

Если в вашей задаче нет пропусков, то возможно, что вам это снится или вы работаете над задачей из параллельного мира, где нет страданий и все счастливы.

Впрочем, если вы все-таки в реальном мире, то (в том или ином виде) вам нужно будет обрабатывать пропуски в данных.

На вскидку можно перечислить несколько вариантов:
- выкинуть те строки\столбцы, где встречаются пропуски;
- заполнить пропуски какими-нибудь значениями по умолчанию (например 0 или -1 для вещественных признаков или "unknown" для категориальных);
- заполнить пропуски статистиками;
- попробовать выделить подпространство без пропусков, посчитать [K-мерное дерево](http://scikit-learn.org/0.19/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree) и по нему уже брать ТОП-(сколько-нибудь) ближайших объектов и уже на них либо считать статистику (среднее или медиана для вещественных, мода - для категориальных);
- и т.д.

К слову сказать, по этому поводу написана целая [книга](http://books.sernam.ru/book_stan.php).

Мы здесь подробно на этой теме останавливаться не будем.

Рассмотрим лишь несколько самых простых подходов, которые могут встретиться вам на практике.

## sklearn.impute.SimpleImputer

Здесь мы заюзаем довольно простой в обращении класс, который предоставляет несколько опций, а именно:
- заполнить пропуски средними значениями;
- заполнить пропуски медианами;
- заполнить пропуски модами;
- заполнить пропуски константами.

Но сначала вспомним из EDA.ipynb, что в данных признаки, в целом, двух типов: вещественные и категориальные. Применим разные стратегии к разным подмножествам признаков:

In [ ]:
numerical_features = [
    'age',
    'campaign',
    'cons.conf.idx',
    'cons.price.idx',
    'duration',
    'emp.var.rate',
    'euribor3m',
    'nr.employed',
    'pdays',
    'previous'
]

In [ ]:
categorial_features = [
    'contact',
    'day_of_week',
    'default',
    'education',
    'housing',
    'job',
    'loan',
    'marital',
    'month',
    'poutcome'
]

In [ ]:
mean_imputer = SimpleImputer(
    missing_values=np.nan,
    strategy='mean',
    verbose=True
)

In [ ]:
median_imputer = SimpleImputer(
    missing_values=np.nan,
    strategy='median',
    verbose=True
)

In [ ]:
most_frequent_imputer = SimpleImputer(
    missing_values=np.nan,
    strategy='most_frequent',
    verbose=True
)

In [ ]:
constant_imputer = SimpleImputer(
    missing_values=np.nan,
    strategy='constant',
    fill_value='unknown',
    verbose=True
)

In [ ]:
numericals_with_mean_imputed = pd.DataFrame(
    mean_imputer.fit_transform(data_with_gaps_v1[numerical_features]),
    columns=numerical_features
)

In [ ]:
numericals_with_mean_imputed.describe().T

In [ ]:
numericals_with_mean_imputed.head().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_mean_imputed)
plt.show()

In [ ]:
numericals_with_median_imputed = pd.DataFrame(
    median_imputer.fit_transform(data_with_gaps_v1[numerical_features]),
    columns=numerical_features
)

In [ ]:
numericals_with_median_imputed.describe().append(
    data_with_gaps_v1[numerical_features].median().rename("median")
).T

In [ ]:
numericals_with_median_imputed.head().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_median_imputed)
plt.show()

In [ ]:
numericals_with_most_frequent_imputed = pd.DataFrame(
    most_frequent_imputer.fit_transform(data_with_gaps_v1[numerical_features]),
    columns=numerical_features
)

In [ ]:
numericals_with_most_frequent_imputed.describe().append(
    data_with_gaps_v1[numerical_features].mode().loc[0].rename("most_frequent")
).T

In [ ]:
numericals_with_most_frequent_imputed.head().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_most_frequent_imputed)
plt.show()

In [ ]:
categorials_with_constant_imputed = pd.DataFrame(
    constant_imputer.fit_transform(data_with_gaps_v1[categorial_features]),
    columns=categorial_features
)

In [ ]:
categorials_with_constant_imputed[categorial_features[0]].value_counts()

In [ ]:
categorials_with_constant_imputed[categorial_features[2]].value_counts()

In [ ]:
categorials_with_constant_imputed[categorial_features[2]].value_counts()

In [ ]:
categorials_with_constant_imputed.head()

# Scalers

Нормализация данных - обычное дело.

Это вещь очень полезная в том случае, если вы используете алгоритмы, чувствительные к масштабу (например метрические алгоритмы).

Здесь мы рассмотрим некоторые из вариантов.

## sklearn.preprocessing.MaxAbsScaler

In [ ]:
max_abs_scaler = MaxAbsScaler()

In [ ]:
numericals_with_mean_imputed_max_abs_scaled = pd.DataFrame(
    max_abs_scaler.fit_transform(numericals_with_mean_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_mean_imputed_max_abs_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_mean_imputed_max_abs_scaled)
plt.show()

In [ ]:
numericals_with_median_imputed_max_abs_scaled = pd.DataFrame(
    max_abs_scaler.fit_transform(numericals_with_median_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_median_imputed_max_abs_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_median_imputed_max_abs_scaled)
plt.show()

In [ ]:
numericals_with_most_frequent_imputed_max_abs_scaled = pd.DataFrame(
    max_abs_scaler.fit_transform(numericals_with_most_frequent_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_most_frequent_imputed_max_abs_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_most_frequent_imputed_max_abs_scaled)
plt.show()

## sklearn.preprocessing.MinMaxScaler

In [ ]:
min_max_scaler = MinMaxScaler()

In [ ]:
numericals_with_mean_imputed_min_max_scaled = pd.DataFrame(
    min_max_scaler.fit_transform(numericals_with_mean_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_mean_imputed_min_max_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_mean_imputed_min_max_scaled)
plt.show()

In [ ]:
numericals_with_median_imputed_min_max_scaled = pd.DataFrame(
    min_max_scaler.fit_transform(numericals_with_median_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_median_imputed_min_max_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_median_imputed_min_max_scaled)
plt.show()

In [ ]:
numericals_with_most_frequent_imputed_min_max_scaled = pd.DataFrame(
    min_max_scaler.fit_transform(numericals_with_most_frequent_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_most_frequent_imputed_min_max_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_most_frequent_imputed_min_max_scaled)
plt.show()

## sklearn.preprocessing.RobustScaler

In [ ]:
robust_scaler = RobustScaler()

In [ ]:
numericals_with_mean_imputed_robust_scaled = pd.DataFrame(
    robust_scaler.fit_transform(numericals_with_mean_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_mean_imputed_robust_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_mean_imputed_robust_scaled)
plt.show()

In [ ]:
numericals_with_median_imputed_robust_scaled = pd.DataFrame(
    robust_scaler.fit_transform(numericals_with_median_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_median_imputed_robust_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_median_imputed_robust_scaled)
plt.show()

In [ ]:
numericals_with_most_frequent_imputed_robust_scaled = pd.DataFrame(
    robust_scaler.fit_transform(numericals_with_most_frequent_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_most_frequent_imputed_robust_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_most_frequent_imputed_robust_scaled)
plt.show()

## sklearn.preprocessing.StandardScaler

In [ ]:
standard_scaler = StandardScaler()

In [ ]:
numericals_with_mean_imputed_standard_scaled = pd.DataFrame(
    standard_scaler.fit_transform(numericals_with_mean_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_mean_imputed_standard_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_mean_imputed_standard_scaled)
plt.show()

In [ ]:
numericals_with_median_imputed_standard_scaled = pd.DataFrame(
    standard_scaler.fit_transform(numericals_with_median_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_median_imputed_standard_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_median_imputed_standard_scaled)
plt.show()

In [ ]:
numericals_with_most_frequent_imputed_standard_scaled = pd.DataFrame(
    standard_scaler.fit_transform(numericals_with_most_frequent_imputed),
    columns=numerical_features
)

In [ ]:
numericals_with_most_frequent_imputed_standard_scaled.describe().T

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_most_frequent_imputed_standard_scaled)
plt.show()

# Transformers

## sklearn.preprocessing.FunctionTransformer

In [ ]:
def all_but_last_column(X):
    return X[:, :-1]

In [ ]:
function_transformer = FunctionTransformer(all_but_last_column)

In [ ]:
numericals_with_median_imputed_standard_scaled_without_last_column = pd.DataFrame(
    function_transformer.fit_transform(numericals_with_median_imputed_standard_scaled),
    columns=numerical_features[:-1]
)

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_median_imputed_standard_scaled_without_last_column)
plt.show()

## sklearn.preprocessing.PowerTransformer

In [ ]:
power_transformer = PowerTransformer()

In [ ]:
numericals_with_median_imputed_standard_scaled_power_transformed = pd.DataFrame(
    power_transformer.fit_transform(numericals_with_median_imputed_standard_scaled),
    columns=numerical_features
)

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_median_imputed_standard_scaled_power_transformed)
plt.show()

## sklearn.preprocessing.QuantileTransformer

In [ ]:
quantile_transformer = QuantileTransformer(output_distribution='normal')

In [ ]:
numericals_with_median_imputed_standard_scaled_quantile_transformed = pd.DataFrame(
    quantile_transformer.fit_transform(numericals_with_median_imputed_standard_scaled),
    columns=numerical_features
)

In [ ]:
plt.figure(figsize=(35, 15))
sns.boxplot(data=numericals_with_median_imputed_standard_scaled_quantile_transformed)
plt.show()

# Encoders

## sklearn.preprocessing.LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
y_encoded = label_encoder.fit_transform(data.y)

In [ ]:
label_encoder.classes_

In [ ]:
y_encoded

In [ ]:
y_decoded = label_encoder.inverse_transform(y_encoded)

In [ ]:
y_decoded

## sklearn.preprocessing.LabelBinarizer

In [ ]:
label_binarizer = LabelBinarizer()

In [ ]:
y_binarized = label_binarizer.fit_transform(data.y)

In [ ]:
label_binarizer.classes_

In [ ]:
label_binarizer.neg_label

In [ ]:
label_binarizer.pos_label

In [ ]:
label_binarizer.y_type_

In [ ]:
y_binarized

In [ ]:
y_original = label_binarizer.inverse_transform(y_binarized)

In [ ]:
y_original

## sklearn.preprocessing.OneHotEncoder

In [ ]:
one_hot_encoder = OneHotEncoder()

In [ ]:
categorials_with_constant_imputed.head().T

In [ ]:
categorials_with_constant_imputed_one_hot_encoded = one_hot_encoder.fit_transform(categorials_with_constant_imputed)

In [ ]:
categorials_with_constant_imputed_one_hot_encoded.shape

In [ ]:
type(categorials_with_constant_imputed_one_hot_encoded)

In [ ]:
categorials_with_constant_imputed_one_hot_encoded.todense()

In [ ]:
categorials_with_constant_imputed_one_hot_encoded = pd.DataFrame(
    categorials_with_constant_imputed_one_hot_encoded.todense(),
    columns=one_hot_encoder.get_feature_names()
)

In [ ]:
categorials_with_constant_imputed_one_hot_encoded.head().T

In [ ]:
one_hot_encoder.categories_

In [ ]:
one_hot_encoder.get_feature_names()

In [ ]:
categorials_with_constant_imputed_one_hot_decoded = one_hot_encoder.inverse_transform(categorials_with_constant_imputed_one_hot_encoded)

In [ ]:
categorials_with_constant_imputed_one_hot_decoded

## sklearn.preprocessing.OrdinalEncoder

In [ ]:
ordinal_encoder = OrdinalEncoder()

In [ ]:
categorials_with_constant_imputed_ordinal_encoded = ordinal_encoder.fit_transform(categorials_with_constant_imputed)

In [ ]:
categorials_with_constant_imputed_ordinal_encoded

In [ ]:
categorials_with_constant_imputed_ordinal_encoded.shape

In [ ]:
ordinal_encoder.categories_

In [ ]:
categorials_with_constant_imputed_ordinal_decoded = ordinal_encoder.inverse_transform(categorials_with_constant_imputed_ordinal_encoded)

In [ ]:
categorials_with_constant_imputed_ordinal_decoded

# Target balancers

## Under-sampling

### imblearn.under_sampling.RandomUnderSampler

In [ ]:
random_undersampler = RandomUnderSampler(random_state=0)

In [ ]:
X_resampled, y_resampled = random_undersampler.fit_resample(
    data[numerical_features + categorial_features],
    data.y
)

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x=y_resampled)
plt.show()

### imblearn.under_sampling.ClusterCentroids

In [ ]:
cluster_centroids_undersampler = ClusterCentroids(random_state=0)

In [ ]:
X_resampled, y_resampled = cluster_centroids_undersampler.fit_resample(
    data[numerical_features],
    data.y
)

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x=y_resampled)
plt.show()

### imblearn.under_sampling.NearMiss

In [ ]:
near_miss_undersampler = NearMiss(random_state=0)

In [ ]:
X_resampled, y_resampled = near_miss_undersampler.fit_resample(
    data[numerical_features],
    data.y
)

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x=y_resampled)
plt.show()

## Over-sampling

### imblearn.over_sampling.RandomOverSampler

In [ ]:
random_oversampler = RandomOverSampler(random_state=0)

In [ ]:
X_resampled, y_resampled = random_oversampler.fit_resample(
    data[numerical_features + categorial_features],
    data.y
)

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x=y_resampled)
plt.show()

### imblearn.over_sampling.SMOTE

In [ ]:
smote_oversampler = SMOTE(random_state=0)

In [ ]:
X_resampled, y_resampled = smote_oversampler.fit_resample(
    data[numerical_features],
    data.y
)

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x=y_resampled)
plt.show()

### imblearn.over_sampling.ADASYN

In [ ]:
adasyn_oversampler = ADASYN(random_state=0)

In [ ]:
X_resampled, y_resampled = adasyn_oversampler.fit_resample(
    data[numerical_features],
    data.y
)

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x=y_resampled)
plt.show()

# A bit about the mess that was created so far

А вот глянем теперь на тот ворох переменных, который был создан:

In [ ]:
list(filter(lambda x: 'categorial' in x or 'numerical' in x, dir()))

# Conclusion

Итак, мы глянули:
- как средствами scikit-learn можно заполнять пропуски, используя разные стратегии;
- как нормализовать данные;
- как с помощью трансформеров проводить преобразования признаков, отсеивать ненужные и т.д.;
- как переводить целевую переменную из текстового представления в OneHot-представление и обратно;
- как переводить категориальные признаки из текстового представления в OneHot-представление и обратно;
- как проводить балансировку классов через:
    - Under-sampling;
    - Over-sampling.

Есть несколько моментов, которые остались непокрытыми, а именно:
- трансформация вещественной переменной в категориальную (квантизация);
- работа с выбросами;
- pipelines;
- hyperparameters search;
- pipelines + hyperparameters search
- multiple metrics calculation during hyperparameters search.

Здесь пока нет смысла делать какие-то выводы по поводу качества предобработки, поскольку мы еще не проводили тестирования на каком бы то ни было алгоритме.

С другой стороны, на этапе различных трансформаций уже довольно четко проглядывалось, что некоторые признаки по виду лучше не становятся (сохраняется асимметрия, никуда не деваются выбросы и т.д.).

К таким признакам стоит присматриваться повнимательнее, возможно там как раз и стоит проводить квантизацию (как, например, в случае с признаком pdays, где за 999 обозначалось отсутствие звонков клиенту).

В целом, конечно, было бы гораздо лучше уже попробовать, какие-нибудь варианты предобработки в комбинации с каким-нибудь алгоритмом с тем, чтобы посмотреть, какие трансформации в какую сторону меняют результат.